# Polebalancing using NESTML

In this tutorial, we are going to build an agent that can successfully solve the classic pole balancing problem using reinforcement learning. We will start with a standard temporal difference learning approach and after that, use NESTML to set up a spiking neural network to perform this task.

# Cart Pole Environment

For the cart pole environment, we mostly need three things:  
    - A renderer to display the simulation  
    - The physics system and  
    - An input to be able to nudge the pole in both directions  

For that, we will need the following packages:

In [1]:
from typing import Tuple

import enum
import json
import matplotlib as mpl
import numpy as np
import os
import time

import pygame as pg

import nest

nest.set_verbosity("M_ERROR")
nest_local_num_threads = 8   # XXX should be 1 by default

mpl.rcParams["axes.grid"] = True
mpl.rcParams["grid.color"] = "k"
mpl.rcParams["grid.linestyle"] = ":"
mpl.rcParams["grid.linewidth"] = 0.5

pygame 2.5.0 (SDL 2.28.0, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html

              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.7.0
 Built: Mar 12 2025 18:15:33

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.



Let's start with the renderer...

In [2]:
class Renderer():
    r"""Renders the physics scene."""
    def __init__(self, width: int, height: int, origin_x: int = 0, origin_y: int = 0, SCALE: int = 1) -> None:
        self.width = width
        self.height = height
        self.origin = (origin_x, origin_y)
        self.SCALE = SCALE #1m = SCALE pixels

        pg.display.init()
        pg.display.set_caption("Pole Balancing Simulator")
        pg.font.init()
        self.screen = pg.display.set_mode((width, height))
    
    #Translates global coordinates into screen coordinates
    def translate(self, x: int, y: int) -> Tuple[int, int]:
        return (x+self.origin[0], -y+self.origin[1])
    
    #Draws ground. offset is there to shift the ground below the car
    def draw_ground(self, offset: int, color) -> None:
        ground = pg.Rect(self.translate(-self.width//2, -offset * self.SCALE), (self.width, self.height-self.origin[1]-offset * self.SCALE))
        pg.draw.rect(self.screen, color, ground)

    #Draws car. pos_y is omitted because the car's center should be at y = 0
    def draw_car(self, pos_x: float, car_color = "blue", wheel_color = "black") -> None:
        pos_x *= self.SCALE
        #values, hard-coded for now, in meters
        width = 0.5 * self.SCALE
        height = 0.25 * self.SCALE
        wheel_radius = 0.1 * self.SCALE

        car_body = pg.Rect(self.translate(pos_x - width/2, height/2), (width, height))
        pg.draw.rect(self.screen, car_color, car_body)
        pg.draw.circle(self.screen, wheel_color, 
                           self.translate(pos_x - width/2 + wheel_radius, -height/2), wheel_radius)
        pg.draw.circle(self.screen, wheel_color, 
                           self.translate(pos_x + width/2 - wheel_radius, -height/2), wheel_radius)

    #Draws the pole
    def draw_pole(self, pos_x: float, theta: float, length: float, width: float = 0.1, color = "red") -> None:
        pos_x *= self.SCALE
        width = int(width * self.SCALE)
        pole_end_x = length * np.sin(theta) * self.SCALE + pos_x
        pole_end_y = length * np.cos(theta) * self.SCALE
        pg.draw.line(self.screen, color, self.translate(pos_x, 0), self.translate(pole_end_x, pole_end_y), width)

    #Clears the entire canvas
    def draw_clear(self) -> None:
        self.screen.fill("white")

    #Draws physical values
    def draw_stats(self, theta: float, w: float, v: float, x: float, 
                    episode: int, 
                    spikes_left : int, spikes_right : int, 
                    dopamine_left : float, dopamine_right : float, 
                    action: int) -> None:
        font = pg.font.Font(None, 24)
        #Physics stats, drawn left
        text = "angle: " + str(theta)[:4] + \
            "\nangular velocity: " + str(w)[:4] + \
            "\nposition: " + str(x)[:4] + \
            "\nvelocity" + str(v)[:4] + \
            " \nepisode: " + str(episode)
        lines = text.split('\n')
        y_pos = 10
        for line in lines:
            text_surface = font.render(line, True, (0,0,0))
            self.screen.blit(text_surface, (10, y_pos))
            y_pos += 30

        #Network stats, drawn right
        text = "Spikes left: " + str(spikes_left) + \
            "\nDopamine left: " + str(dopamine_left)[:4] + \
            "\nSpikes right: " + str(spikes_right)[:4] + \
            "\nDopamine right: " + str(dopamine_right) + \
            "\nTaken action: " + ("Left" if action==AgentAction.LEFT else "Right" if action==AgentAction.RIGHT else "Failure")
        lines = text.split('\n')
        y_pos = 10
        for line in lines:
            text_surface = font.render(line, True, (0,0,0))
            self.screen.blit(text_surface, (self.width - 200, y_pos))
            y_pos += 30

    def get_relative_mouse_x(self, mouse_x:float) -> float:
        return (mouse_x-self.origin[0])/self.SCALE
    
    def display(self) -> None:
        pg.display.flip()

## Physics Updates

For the physics, we use the corrected version of of the original problem derived from V. Florian (CITATION NEEDED), but omit the friction forces.
The situation is sketched here:  

![alt text](cartpole_illustration.png "Cartpole")

We apply Newton's second law of motion to the cart:  
$$
\begin{aligned}
    \mathbf{F} + \mathbf{G}_c - \mathbf{N} = m_c \cdot \mathbf{a}_c
\end{aligned}
$$
Where:  

$\mathbf{F} = F \cdot \mathbf{u_x}$ is the control force acting on the cart,  
$\mathbf{G}_c = m_c \cdot g \cdot \mathbf{u}_y$ is the gravitational component acting on the cart,  
$\mathbf{N} = N_x \cdot \mathbf{u}_x - N_y \cdot \mathbf{u}_y$ is the negative reaction force that the pole is applying on the cart,  
$\mathbf{a}_c = \ddot{x} \cdot \mathbf{u}_x$ is the accelaration of the cart,  
$m_c$ is the cart's mass and  
$\mathbf{u}_x$, $\mathbf{u}_y$, $\mathbf{u}_z$ are the unit vectors of the frame of reference given in the illustration.

We can decompose this equation now into the $x$ and $y$ component:
$$
\begin{aligned}
    F - N_x = m_c \cdot \ddot{x}
\end{aligned}
$$
$$
\begin{aligned}
    m_c \cdot g + N_y = 0
\end{aligned}
$$

Newton's second law of motion applied to the pole gives us:
$$
\begin{aligned}
    \mathbf{N} + \mathbf{G}_p = m_p \cdot \mathbf{a}_p
\end{aligned}
$$

Where $\mathbf{G}_p = m_p \cdot g \cdot \mathbf{u}_y$.

The accelaration $\mathbf{a}_p$ of the pole's center of mass consists of three components, where $\mathbf{r}_p = l \cdot (\sin{\theta}\cdot \mathbf{u}_x-\cos{\theta}\cdot \mathbf{u}_y)$ denotes the vector pointing to the pole's center of mass relative to it's rotation center:  
1. The accelaration of the cart it is attached to $\mathbf{a}_c$,
2. The pole's angular accelaration $\mathbf{\epsilon} = \ddot{\theta} \cdot \mathbf{u}_z$, which is translated into accelaration by $\mathbf{\epsilon} \times \mathbf{r}_p$.
3. The pole's angular velocity $\mathbf{\omega} = \dot{\theta} \cdot \mathbf{u}_z$, for which the accelaration can be derived by  $\mathbf{\omega} \times (\mathbf{\omega} \times \mathbf{r}_p)$.

Thus we obtain:
$$
\begin{aligned}
    \mathbf{a}_p  = \mathbf{a}_c + \mathbf{\epsilon} \times \mathbf{r}_p + \mathbf{\omega} \times (\mathbf{\omega} \times \mathbf{r}_p)
\end{aligned}
$$
Substituting $\mathbf{r}_p = l \cdot (\sin{\theta}\cdot \mathbf{u}_x-\cos{\theta}\cdot \mathbf{u}_y)$ and $\mathbf{a}_p = \ddot{x} \cdot \mathbf{u}_x$ as well as $\mathbf{u}_z \times \mathbf{u}_x = \mathbf{u}_y$ and $\mathbf{u}_z \times \mathbf{u}_y = -\mathbf{u}_x$:
\begin{aligned}
    \mathbf{a}_p  = \ddot{x} \cdot \mathbf{u}_x + l \cdot \ddot{\theta} \cdot (\sin{\theta}\cdot \mathbf{u}_y + \cos{\theta}\cdot \mathbf{u}_x) - l \cdot \dot{\theta}^2 \cdot (\sin{\theta}\cdot \mathbf{u}_x - \cos{\theta}\cdot \mathbf{u}_y)
\end{aligned}

Inserting this quation into our equation for the forces of the pole and decomposing on the $x$ and $y$ axis we obtain:
$$
\begin{aligned}
    N_x = m_p \cdot (\ddot{x} + l \cdot \ddot{\theta} \cdot \cos{\theta} - l \cdot \dot{\theta}^2 \cdot \sin{\theta})
\end{aligned}
$$
$$
\begin{aligned}
    m_p \cdot g - N_y = m_p \cdot (l \cdot \ddot{\theta} \cdot \sin{\theta} + l \cdot \dot{\theta}^2 \cdot \cos{\theta})
\end{aligned}
$$

# TODO: FINISH EQUATION DERIVATION (SOLVE EQUATION REFERENCING?)

In [3]:
class Physics:
    
    def __init__(self, x, theta, v = 0, a = 0, w = 0, dw = 0, g = 9.81, m_c = 1, m_p = 0.1, l = 0.5, dt = 0.02) -> None:
        self.__dict__.update(vars())

    def dw_step(self, cart_force, nudge_force) -> float:
        numerator = self.g * np.sin(self.theta) + np.cos(self.theta) * (-cart_force - self.m_p * self.l * self.w**2 * np.sin(self.theta))/(self.m_c+self.m_p) + nudge_force * np.cos(self.theta)/(self.m_p*self.l)
        denominator = self.l * (4/3 - (self.m_p*np.cos(self.theta)**2)/(self.m_c+self.m_p))

        self.dw = numerator/denominator
        self.w += self.dt * self.dw
        self.theta += self.dt * self.w

        return self.theta
    
    def a_step(self, force) -> float:
        numerator = force + self.m_p * self.l * (self.w**2 * np.sin(self.theta) - self.dw * np.cos(self.theta))
        denominator = self.m_c + self.m_p

        self.a = numerator/denominator
        self.v += self.dt * self.a
        self.x += self.dt * self.v

        return self.x

    def update(self, force, mouse_x) -> Tuple[float, float]:
        nudge_force = 0
        if mouse_x is not None:
            nudge_force = -1 if mouse_x > self.x else 1
        return (self.dw_step(force, nudge_force), self.a_step(force))
    
    #get state of the system that agent can see
    def get_state(self) -> Tuple[float,float,float,float]:
        return (self.x, self.theta, self.v, self.w)
    
    def reset(self) -> None:
        self.x = 0
        self.theta = (np.random.rand() - 1) / 10
        self.v = 0
        self.a = 0
        self.w = 0
        self.dw = 0


# The Agent (BOXES)

In [4]:

class Agent:
    """
    State is represented as:

    [ x ]
    [ θ ]
    [ v ]
    [ ω ]   # = dθ/dt

    """
    
    def __init__(self, initial_state: Tuple[float,float,float,float]) -> None:

        #thresholds for discretizing the state space
        
        # ORIGINAL BOXES THAT WE USED SUCCESSFULLY ON THE NON SPIKING AGENT
        self.x_thresholds = np.array([-2.4, -0.8, 0.8, 2.4])
        self.theta_thresholds = np.array([-12, -6, -1, 0, 1, 6, 12])
        self.theta_thresholds = self.theta_thresholds /180 * np.pi
        self.v_thresholds = np.array([float("-inf"), -0.5, 0.5, float("+inf")]) #open intervals ignored here
        self.w_thresholds = np.array([float("-inf"), -50, 50, float("+inf")]) #open intervals ignored here
        self.w_thresholds = self.w_thresholds / 180 * np.pi

#         # BOXES FROM LIU&PAN CODE
#         self.x_thresholds = np.array([-2.4, 2.4])
#         self.v_thresholds = np.array([float("-inf"), float("+inf")])
        
#         self.theta_thresholds = np.array([-12, -5.738738738738739, -2.8758758758758756, 0., 2.8758758758758756, 5.738738738738739, 12])
#         self.theta_thresholds = self.theta_thresholds / 180 * np.pi
        
#         self.w_thresholds = np.array([float("-inf"), -103., -91.7, -80.2, -68.8, -57.3, -45.9, -34.3, -22.9, -11.5, 0.,
#                                                       11.5, 22.9, 34.3, 45.9, 57.3, 68.8, 80.2, 91.7, 103., float("+inf")]) #open intervals ignored here
#         self.w_thresholds = self.w_thresholds / 180 * np.pi
        
        self.dimensions = (len(self.x_thresholds) - 1, len(self.theta_thresholds) - 1, len(self.v_thresholds) - 1, len(self.w_thresholds) - 1)

        print("Dimension of input space: " + str(self.dimensions))
        
        self.boxes = np.random.rand(self.dimensions[0], 
                                    self.dimensions[1], 
                                    self.dimensions[2], 
                                    self.dimensions[3], 
                                    2) #one q-value for left and right respectively
        box = self.get_box(initial_state)
        self.current_box = self.boxes[box[0], box[1], box[2], box[3], :]

        self.episode = 1
    
    def discretize(self, value, thresholds):
        for i, limit in enumerate(thresholds):
            if value < limit:
                return i - 1
        return -1

    def get_box(self, state: Tuple[float,float,float,float]) -> Tuple[int,int,int,int]:
        return (self.discretize(state[0], self.x_thresholds),
                 self.discretize(state[1], self.theta_thresholds),
                 self.discretize(state[2], self.v_thresholds), 
                 self.discretize(state[3], self.w_thresholds))
    
    def get_episode(self) -> int:
        return self.episode
    
    
    def failure_reset(self, state: Tuple[float,float,float,float]):
        box = self.get_box(state)
        self.current_box = self.boxes[box[0], box[1], box[2], box[3], :]
        self.episode += 1


class NonSpikingAgent(Agent):
    def __init__(self, initial_state: Tuple[float,float,float,float], learning_rate, learning_decay, epsilon, epsilon_decay, discount_factor) -> None:
        super().__init__(initial_state)

        #learning paramters
        self.learning_rate = learning_rate
        self. learning_decay = learning_decay
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.discount_factor = discount_factor

    #returns 0 if the action is "left", else "1"
    def choose_action(self) -> int:
        self.action = np.random.choice([np.argmax(self.current_box), np.argmin(self.current_box)], p=[1-self.epsilon, self.epsilon])
        return self.action
    
    #returns 0 if no failure occured, else 1
    #reward is -1 on failure and 0 else
    def update(self, next_state: Tuple[float,float,float,float]) -> int:
        box = self.get_box(next_state)
        if -1 in box:
            self.current_box[self.action] += self.learning_rate * -1
            return 1
        
        next_box = self.boxes[box[0], box[1], box[2], box[3], :]
        next_q = np.max(next_box)
        self.current_box[self.action] += self.learning_rate * (self.discount_factor * (next_q - self.current_box[self.action]))

        self.current_box = next_box
        self.epsilon *= self.epsilon_decay
        self.learning_rate *= self.learning_decay

        return 0
    

# Plot Renderer

In [5]:
import matplotlib.pyplot as plt
%matplotlib qt
class Non_Spiking_PlotRenderer():
    def __init__(self, init_x = [0], init_y = [0]) -> None:
        plt.ion()

        #Construct lifetime plot
        self.lifetime_fig, self.lifetime_ax = plt.subplots()
        self.x = init_x
        self.y = init_y
        self.max_lifetime = 0
        self.lifetime_line, = self.lifetime_ax.plot(self.x, self.y)
        self.lifetime_ax.set_xlabel("Episode")
        self.lifetime_ax.set_ylabel("Simulation Steps")
        self.lifetime_ax.set_title("Lifetime Plot")

        #Construct Heatmap for two parameters
        self.q_value_fig, self.q_value_ax = plt.subplots()
        self.q_value_ax.set_title("Q-Values for a state of (param1/param2)")
        self.cmap = plt.cm.coolwarm
        
    def update(self, x, y, boxes) -> None:
        self.x.append(x)
        self.y.append(y)
        self.max_lifetime = max(self.max_lifetime, y)
        self.lifetime_line.set_data(self.x, self.y)
        self.lifetime_ax.set_xlim(self.x[0], self.x[-1])
        self.lifetime_ax.set_ylim(0, self.max_lifetime)

        if(x % 10 == 0):
            q_values = boxes[:,:,:,:,0] - boxes[:,:,:,:,1]
            self.q_value_ax.imshow(np.mean(q_values, axis = (1,3)), cmap=self.cmap, interpolation='none')

        plt.draw()
        plt.pause(0.0001)



# Executing Non-Spiking-Agent

In [6]:
"""

import sys

r = Renderer(1200, 800, 600, 500, 400)
clock = pg.time.Clock()
running = True

p = Physics(0, (np.random.rand() - 1) / 10)

a = NonSpikingAgent(p.get_state(), 0.5, 0.9999999999999, 1, 0.995, 0.99)

plot = Non_Spiking_PlotRenderer()

steps_per_episode = 0
max_steps = 0

window_size = 30
window = np.zeros(30)
avg_lifetime = 20000

toggle_sim = False

while running:
    steps_per_episode += 1

    force = 0
    mouse_x = None

    # poll for events
    for event in pg.event.get():
        if event.type == pg.QUIT:
            running = False
            pg.quit()
            sys.exit()
            quit()
        elif event.type == pg.MOUSEBUTTONDOWN:
            mouse_x = r.get_relative_mouse_x(pg.mouse.get_pos()[0])
        elif event.type == pg.KEYDOWN:
            toggle_sim ^= pg.key.get_pressed()[pg.K_SPACE]

    # agent chooses action, simulation is updated and reward is calculated
    force = 10 if a.choose_action() else -10
    theta, x = p.update(force, mouse_x)
    failure = a.update(p.get_state())

    if failure:
        p.reset()
        a.failure_reset(p.get_state())
        plot.update(a.get_episode(), steps_per_episode, a.boxes)
        window = np.roll(window, 1)
        window[0] = steps_per_episode
        steps_per_episode = 0
    
    
    if np.mean(window) >= avg_lifetime or toggle_sim:
        r.draw_clear()
        r.draw_ground(0.2, "grey")
        r.draw_car(x)
        r.draw_pole(x, theta, 2*p.l, 0.02)
        r.draw_stats(theta*180/np.pi, p.w*180/np.pi, x, p.a, a.get_episode())
        r.display()

        clock.tick(50)  # limits FPS to 50
"""

'\n\nimport sys\n\nr = Renderer(1200, 800, 600, 500, 400)\nclock = pg.time.Clock()\nrunning = True\n\np = Physics(0, (np.random.rand() - 1) / 10)\n\na = NonSpikingAgent(p.get_state(), 0.5, 0.9999999999999, 1, 0.995, 0.99)\n\nplot = Non_Spiking_PlotRenderer()\n\nsteps_per_episode = 0\nmax_steps = 0\n\nwindow_size = 30\nwindow = np.zeros(30)\navg_lifetime = 20000\n\ntoggle_sim = False\n\nwhile running:\n    steps_per_episode += 1\n\n    force = 0\n    mouse_x = None\n\n    # poll for events\n    for event in pg.event.get():\n        if event.type == pg.QUIT:\n            running = False\n            pg.quit()\n            sys.exit()\n            quit()\n        elif event.type == pg.MOUSEBUTTONDOWN:\n            mouse_x = r.get_relative_mouse_x(pg.mouse.get_pos()[0])\n        elif event.type == pg.KEYDOWN:\n            toggle_sim ^= pg.key.get_pressed()[pg.K_SPACE]\n\n    # agent chooses action, simulation is updated and reward is calculated\n    force = 10 if a.choose_action() else -10\

# TODO: clean up code, derive equations and explain renderer briefly

In [7]:
def create_weight_matrix(connections):
    """
    Create a weight matrix from NEST connections.
    
    Parameters:
    -----------
    connections : nest.NodeCollection
        Connection object obtained from nest.GetConnections()
        
    Returns:
    --------
    weight_matrix : numpy.ndarray
        Matrix of shape (n_pre, n_post) containing connection weights
    """
    # Get connection properties
    conn_info = connections.get(["source", "target", "weight"])
    
    # Extract source, target, and weight arrays
    sources = np.array(conn_info["source"])
    targets = np.array(conn_info["target"])
    weights = np.array(conn_info["weight"])
    
    # Get unique pre and post neuron IDs
    pre_neurons = np.unique(sources)
    post_neurons = np.unique(targets)
    
    # Create a mapping from neuron IDs to matrix indices
    pre_map = {neuron: i for i, neuron in enumerate(pre_neurons)}
    post_map = {neuron: i for i, neuron in enumerate(post_neurons)}
    
    # Initialize weight matrix with zeros
    n_pre = len(pre_neurons)
    n_post = len(post_neurons)
    weight_matrix = np.zeros((n_pre, n_post))
    
    # Fill the weight matrix
    for src, tgt, w in zip(sources, targets, weights):
        pre_idx = pre_map[src]
        post_idx = post_map[tgt]
        weight_matrix[pre_idx, post_idx] = w
    
    return weight_matrix, pre_neurons, post_neurons


# Spiking version

## Idea

The core principle of our SNN is to simulate the physics and neuron model in sequence, where the state at the end of a physics step is the input for the SNN and the resulting action at the end of a period of SNN simulation is the input to the next physics simulation. Both cycles are set to 40ms to provide the effect that they run simultaneously.
The model's structure consists of two layers of neurons. For each discrete state of the system, the input layer contains a single neuron corresponding to it. Neuromodulated synapses connect these to the output layer, which itself consists of two neuron groups interpreted as actions "move left" and "move right" respectively.

One simulation step of the SNN works as follows:
1. Get the current state of the cart pole and find the designated neuron that only fires when that state is reached.
2. Set a continuous firing rate for the simulation period on that neuron.
3. Determine which of the neuron groups in the output layer has fired more spikes at the end of the step.

# SNN Visualization

In [8]:
class Spiking_PlotRenderer:
    def __init__(self) -> None:

#         plt.ion()
        
        self.fig, self.ax = plt.subplots(nrows=6, figsize=(12, 10))
#         self.fig.show()
        
        # Construct lifetime plot
        self.lifetime_fig, self.lifetime_ax = plt.subplots()
        self.lifetime_line, = self.lifetime_ax.plot([0,1], [0,1])
        self.lifetime_ax.set_xlabel("Episode")
        self.lifetime_ax.set_ylabel("Simulation Steps")
        self.lifetime_ax.set_title("Lifetime Plot")
        # self.lifetime_fig.show()
        
        # Plot the weights
        self.weights_fig, self.weights_ax = plt.subplots(ncols=2)
        self.weights_ax[0].set_title("Weights left")
        self.weights_ax[1].set_title("Weights right")
        self.cmap = plt.cm.coolwarm
        self.cbar = None
        
        # plot actions taken
        

    def update(self, data) -> None:
        if data is None: return

        self.weights_ax[0].cla()
        self.weights_ax[1].cla()
#         weights_left, _, _ = create_weight_matrix(syn_left)#np.array(data["weights_left"]).reshape((len(data["weights_left"])//10, 10))
#         weights_right, _, _ = create_weight_matrix(syn_left)#np.array(data["weights_left"]).reshape((len(data["weights_left"])//10, 10))
#         weights_right = np.array(data["weights_right"]).reshape((len(data["weights_right"])//10, 10))

        vmin = min(data["weights_left"].min(), data["weights_right"].min())
        vmax = max(data["weights_left"].max(), data["weights_right"].max())
        norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)

        im_left = self.weights_ax[0].imshow(data["weights_left"], cmap=self.cmap, interpolation='none', norm=norm)
        im_right = self.weights_ax[1].imshow(data["weights_right"], cmap=self.cmap, interpolation='none', norm=norm)

        if not self.cbar:
            self.cbar = self.weights_fig.colorbar(im_left, ax=self.weights_ax, orientation='vertical', pad=0.01)
        
        self.cbar.update_normal(im_left)
        
        
        
        
        
        self.ax[0].cla()
        self.ax[1].cla()
        self.ax[2].cla()
        self.ax[3].cla()
        self.ax[4].cla()
        self.ax[5].cla()

        # Top plot for spikes
        self.ax[0].set_ylabel("Input Neuron")
        self.ax[0].set_ylim(0, data["n_input_neurons"])
        self.ax[0].plot(data["input_spikes"]["times"], data["input_spikes"]["senders"], ".k", markersize=5)
        
#         self.ax[1].plot(data["multimeter_right_events"]["times"], data["multimeter_right_events"]["V_m"], 'r')
#         self.ax[1].plot(data["multimeter_left_events"]["times"], data["multimeter_left_events"]["V_m"], 'b')

        for neuron_id in np.unique(data["multimeter_left_events"]["senders"]):
            idx = np.where(data["multimeter_left_events"]["senders"] == neuron_id)[0]
            neuron_times = data["multimeter_left_events"]["times"][idx]
            neuron_V_m = data["multimeter_left_events"]["V_m"][idx]
            self.ax[1].plot(neuron_times, neuron_V_m, c="b")

        for neuron_id in np.unique(data["multimeter_right_events"]["senders"]):
            idx = np.where(data["multimeter_right_events"]["senders"] == neuron_id)[0]
            neuron_times = data["multimeter_right_events"]["times"][idx]
            neuron_V_m = data["multimeter_right_events"]["V_m"][idx]
            self.ax[1].plot(neuron_times, neuron_V_m, c="r")
            
        self.ax[1].set_ylabel("V_m [mV]")
        
        self.ax[2].plot(data["output_spikes_left"]["times"], data["output_spikes_left"]["senders"], ".b", markersize=5)
        self.ax[2].plot(data["output_spikes_right"]["times"], data["output_spikes_right"]["senders"], ".r", markersize=5)
        self.ax[2].set_ylabel("Output Neuron")
        
        self.ax[3].plot(data["action_taken_times"], [action.value for action in data["action_taken"]], "k")
        self.ax[3].set_ylabel("Action taken")
        self.ax[3].set_yticks([AgentAction.LEFT.value, AgentAction.RIGHT.value])
        self.ax[3].set_yticklabels(["LEFT", "RIGHT"])
        
        self.ax[4].plot(data["action_taken_times"], data["p_explore_log"], "k")
#         print("action_taken_times = " + str(data["action_taken_times"]))
#         print("p_explore_log = " + str(data["p_explore_log"]))
        self.ax[4].set_ylabel("$p_\mathrm{explore}$")
        self.ax[4].set_ylim(0, 1)
        
        self.ax[5].plot(data["action_taken_times"], data["epoch_log"], "k")
#         print("action_taken_times = " + str(data["action_taken_times"]))
#         print("p_explore_log = " + str(data["p_explore_log"]))
        self.ax[5].set_ylabel("Epoch")
        
        for _ax in self.ax:
            _ax.set_xlim(np.min(data["multimeter_right_events"]["times"]), np.max(data["multimeter_right_events"]["times"]))
            if not _ax == self.ax[-1]:
                _ax.set_xticklabels([])
        
        self.ax[-1].set_xlabel("Time [ms]")
                
            
            
            
            
        max_lifetime = np.amax(np.array(data["steps_per_episode_log"]) * data["episode_duration"])
        self.lifetime_line.set_data(data["episode_number_log"], np.array(data["steps_per_episode_log"]) * data["episode_duration"])
        self.lifetime_ax.set_xlim(data["episode_number_log"][0], data["episode_number_log"][-1])
        self.lifetime_ax.set_ylim(0, max_lifetime)
        self.lifetime_ax.set_ylabel("Longevity [ms]")
        
        self.ax[-1].set_xlabel("Time [ms]")
        
#         self.fig.canvas.draw()
        self.fig.savefig("/tmp/cartpole.png", dpi=300)
#         self.fig.canvas.flush_events()
        
#         self.lifetime_fig.canvas.draw()
        self.lifetime_fig.savefig("/tmp/cartpole_lifetime.png", dpi=300)
#         self.lifetime_fig.canvas.flush_events()
        
#         self.weights_fig.canvas.draw()
        self.weights_fig.savefig("/tmp/cartpole_weights.png", dpi=300)
#         self.weights_fig.canvas.flush_events()
        
#         plt.pause(0.0001)

## Neuron Models

### Ignore and Fire Neuron

In [9]:
# ... generate NESTML model code...

from pynestml.codegeneration.nest_code_generator_utils import NESTCodeGeneratorUtils

# generate and build code
input_layer_module_name, input_layer_neuron_model_name = \
   NESTCodeGeneratorUtils.generate_code_for("../../../models/neurons/ignore_and_fire_neuron.nestml")

# ignore_and_fire
output_layer_module_name, output_layer_neuron_model_name, output_layer_synapse_model_name = \
    NESTCodeGeneratorUtils.generate_code_for("iaf_psc_exp_neuron.nestml",
                                             "neuromodulated_stdp_synapse.nestml",
                                             post_ports=["post_spikes"],
                                             logging_level="DEBUG",
                                             codegen_opts={"delay_variable": {"neuromodulated_stdp_synapse": "d"},
                                                           "weight_variable": {"neuromodulated_stdp_synapse": "w"}})



              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.7.0
 Built: Mar 12 2025 18:15:33

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.

[12,ignore_and_fire_neuron_nestml, WARNING, [35:34;35:58]]: Model contains a call to fixed-timestep functions (``resolution()`` and/or ``steps()``). This restricts the model to being compatible only with fixed-timestep simulators. Consider eliminating ``resolution()`` and ``steps()`` from the model, and using ``timestep()`` instead.
[1,GLOBAL, INFO]: List of files that will be processed:
[2,GLOBAL, INFO]: /home/charl/julich/nestml-fork-AlexisWis-cart_pole_tutorial-stash-restore/nestml/doc/tutorials/cart_pole_reinforcement_learning/iaf_psc_exp_neuron.nestml
[3,GLOBAL, INFO]: /home/charl/julich/nestml-fork-AlexisWis-cart_pole_tutorial-stash-restore/nestml/doc/tuto

INFO:Analysing input:
INFO:{
    "dynamics": [
        {
            "expression": "g_e' = (-g_e) / tau_g",
            "initial_values": {
                "g_e": "0.0"
            }
        },
        {
            "expression": "V_m' = (g_e * (E_e - V_m) + E_l - V_m + I_e + I_stim) / tau_m",
            "initial_values": {
                "V_m": "E_l"
            }
        }
    ],
    "options": {
        "output_timestep_symbol": "__h"
    },
    "parameters": {
        "E_e": "0",
        "E_l": "(-74)",
        "I_e": "0",
        "V_reset": "(-60)",
        "V_th": "(-54)",
        "s": "1000",
        "tau_g": "5",
        "tau_m": "10"
    }
}
INFO:Processing global options...
INFO:Processing input shapes...
INFO:
Processing differential-equation form shape g_e with defining expression = "(-g_e) / tau_g"
DEBUG:Splitting expression -g_e/tau_g (symbols [g_e])
DEBUG:	linear factors: Matrix([[-1/tau_g]])
DEBUG:	inhomogeneous term: 0.0
DEBUG:	nonlinear term: 0.0
DEBUG:Created Shape

[21,GLOBAL, INFO]: State variables that will be moved from synapse to neuron: ['post_trace']
[22,GLOBAL, INFO]: Parameters that will be copied from synapse to neuron: ['tau_tr_post']
[23,GLOBAL, INFO]: Synaptic state variables moved to neuron that will need buffering: []
[24,GLOBAL, INFO]: Moving state var defining equation(s) post_trace
[25,GLOBAL, INFO]: Moving state variables for equation(s) post_trace
[26,GLOBAL, INFO]: Moving definition of post_trace from synapse to neuron
[27,GLOBAL, INFO]: 	Moving statement post_trace += -1.05e-07 # XXX FIXME!!!! should be ``+= post_trace_increment``
[28,GLOBAL, INFO]: In synapse: replacing ``continuous`` type input ports that are connected to postsynaptic neuron with external variable references
[29,GLOBAL, INFO]: Copying parameters from synapse to neuron...
[30,GLOBAL, INFO]: Copying definition of tau_tr_post from synapse to neuron
[31,GLOBAL, INFO]: Adding suffix to variables in spike updates
[32,GLOBAL, INFO]: In synapse: replacing variables

DEBUG:	nonlinear term: E_e*g_e/tau_m + I_stim/tau_m - V_m*g_e/tau_m
DEBUG:Created Shape with symbol V_m, derivative_factors = [-1/tau_m], inhom_term = E_l/tau_m + I_e/tau_m, nonlin_term = E_e*g_e/tau_m + I_stim/tau_m - V_m*g_e/tau_m
INFO:	Returning shape: Shape "V_m" of order 1
INFO:Shape V_m: reconstituting expression E_e*g_e/tau_m + E_l/tau_m + I_e/tau_m + I_stim/tau_m - V_m*g_e/tau_m - V_m/tau_m
INFO:All known variables: [g_e, V_m], all parameters used in ODEs: {tau_m, I_stim, E_e, E_l, I_e, tau_g}
INFO:No numerical value specified for parameter "I_stim"
INFO:
Processing differential-equation form shape g_e with defining expression = "(-g_e) / tau_g"
DEBUG:Splitting expression -g_e/tau_g (symbols [g_e, V_m, g_e])
DEBUG:	linear factors: Matrix([[-1/tau_g], [0], [0]])
DEBUG:	inhomogeneous term: 0.0
DEBUG:	nonlinear term: 0.0
DEBUG:Created Shape with symbol g_e, derivative_factors = [-1/tau_g], inhom_term = 0.0, nonlin_term = 0
INFO:	Returning shape: Shape "g_e" of order 1
INFO:
Proces

DEBUG:Splitting expression (E_l + I_e + I_stim - V_m + g_e*(E_e - V_m))/tau_m (symbols [g_e, V_m, g_e, V_m])
DEBUG:	linear factors: Matrix([[E_e/tau_m], [-1/tau_m], [0], [0]])
DEBUG:	inhomogeneous term: E_l/tau_m + I_e/tau_m + I_stim/tau_m
DEBUG:	nonlinear term: -V_m*g_e/tau_m
DEBUG:Created Shape with symbol V_m, derivative_factors = [-1/tau_m], inhom_term = E_l/tau_m + I_e/tau_m + I_stim/tau_m, nonlin_term = E_e*g_e/tau_m - V_m*g_e/tau_m
INFO:	Returning shape: Shape "V_m" of order 1
INFO:Shape g_e: reconstituting expression -g_e/tau_g
DEBUG:Splitting expression -g_e/tau_g (symbols Matrix([[g_e], [V_m]]))
DEBUG:	linear factors: Matrix([[-1/tau_g], [0]])
DEBUG:	inhomogeneous term: 0.0
DEBUG:	nonlinear term: 0.0
INFO:Shape V_m: reconstituting expression E_e*g_e/tau_m + E_l/tau_m + I_e/tau_m + I_stim/tau_m - V_m*g_e/tau_m - V_m/tau_m
DEBUG:Splitting expression E_e*g_e/tau_m + E_l/tau_m + I_e/tau_m + I_stim/tau_m - V_m*g_e/tau_m - V_m/tau_m (symbols Matrix([[g_e], [V_m]]))
DEBUG:	linear fa

...

INFO:	Returning shape: Shape "post_trace__for_neuromodulated_stdp_synapse_nestml" of order 1
INFO:Shape g_e: reconstituting expression -g_e/tau_g
DEBUG:Splitting expression -g_e/tau_g (symbols Matrix([[g_e], [V_m], [post_trace__for_neuromodulated_stdp_synapse_nestml]]))
DEBUG:	linear factors: Matrix([[-1/tau_g], [0], [0]])
DEBUG:	inhomogeneous term: 0.0
DEBUG:	nonlinear term: 0.0
INFO:Shape V_m: reconstituting expression E_e*g_e/tau_m + E_l/tau_m + I_e/tau_m + I_stim/tau_m - V_m*g_e/tau_m - V_m/tau_m
DEBUG:Splitting expression E_e*g_e/tau_m + E_l/tau_m + I_e/tau_m + I_stim/tau_m - V_m*g_e/tau_m - V_m/tau_m (symbols Matrix([[g_e], [V_m], [post_trace__for_neuromodulated_stdp_synapse_nestml]]))
DEBUG:	linear factors: Matrix([[E_e/tau_m], [-1/tau_m], [0]])
DEBUG:	inhomogeneous term: E_l/tau_m + I_e/tau_m + I_stim/tau_m
DEBUG:	nonlinear term: -V_m*g_e/tau_m
INFO:Shape post_trace__for_neuromodulated_stdp_synapse_nestml: reconstituting expression -post_trace__for_neuromodulated_stdp_synapse_n

DEBUG:	linear factors: Matrix([[-1/tau_tr_pre]])
DEBUG:	inhomogeneous term: 0.0
DEBUG:	nonlinear term: 0.0
INFO:Saving dependency graph plot to /tmp/ode_dependency_graph_analytically_solvable_before_propagated.dot
DEBUG:os.makedirs('/tmp')
DEBUG:write lines to '/tmp/ode_dependency_graph_analytically_solvable_before_propagated.dot'
DEBUG:run [PosixPath('dot'), '-Kdot', '-Tpdf', '-O', 'ode_dependency_graph_analytically_solvable_before_propagated.dot']
INFO:Saving dependency graph plot to /tmp/ode_dependency_graph_analytically_solvable.dot
DEBUG:os.makedirs('/tmp')
DEBUG:write lines to '/tmp/ode_dependency_graph_analytically_solvable.dot'
DEBUG:run [PosixPath('dot'), '-Kdot', '-Tpdf', '-O', 'ode_dependency_graph_analytically_solvable.dot']
INFO:Generating propagators for the following symbols: pre_trace
DEBUG:Initializing system of shapes with x = Matrix([[pre_trace]]), A = Matrix([[-1/tau_tr_pre]]), b = Matrix([[0]]), c = Matrix([[0]])
DEBUG:System of equations:
DEBUG:x = Matrix([[pre_tr

[39,GLOBAL, INFO]: Analysing/transforming synapse neuromodulated_stdp_synapse_nestml__with_iaf_psc_exp_neuron_nestml.
[40,neuromodulated_stdp_synapse_nestml__with_iaf_psc_exp_neuron_nestml, INFO, [6:0;53:0]]: Starts processing of the model 'neuromodulated_stdp_synapse_nestml__with_iaf_psc_exp_neuron_nestml'
[41,GLOBAL, INFO]: Rendering template /home/charl/julich/nestml-fork-AlexisWis-cart_pole_tutorial-stash-restore/nestml/doc/tutorials/cart_pole_reinforcement_learning/target/iaf_psc_exp_neuron_nestml.cpp
[42,GLOBAL, INFO]: Rendering template /home/charl/julich/nestml-fork-AlexisWis-cart_pole_tutorial-stash-restore/nestml/doc/tutorials/cart_pole_reinforcement_learning/target/iaf_psc_exp_neuron_nestml.h
[43,iaf_psc_exp_neuron_nestml, INFO, [18:0;58:0]]: Successfully generated code for the model: 'iaf_psc_exp_neuron_nestml' in: '/home/charl/julich/nestml-fork-AlexisWis-cart_pole_tutorial-stash-restore/nestml/doc/tutorials/cart_pole_reinforcement_learning/target' !
[44,GLOBAL, INFO]: Ren

/home/charl/julich/nestml-fork-AlexisWis-cart_pole_tutorial-stash-restore/nestml/doc/tutorials/cart_pole_reinforcement_learning/target/neuromodulated_stdp_synapse_nestml__with_iaf_psc_exp_neuron_nestml.h: In instantiation of ‘void nest::neuromodulated_stdp_synapse_nestml__with_iaf_psc_exp_neuron_nestml<targetidentifierT>::recompute_internal_variables() [with targetidentifierT = nest::TargetIdentifierPtrRport]’:
/home/charl/julich/nestml-fork-AlexisWis-cart_pole_tutorial-stash-restore/nestml/doc/tutorials/cart_pole_reinforcement_learning/target/neuromodulated_stdp_synapse_nestml__with_iaf_psc_exp_neuron_nestml.h:741:3:   required from ‘nest::neuromodulated_stdp_synapse_nestml__with_iaf_psc_exp_neuron_nestml<targetidentifierT>::neuromodulated_stdp_synapse_nestml__with_iaf_psc_exp_neuron_nestml() [with targetidentifierT = nest::TargetIdentifierPtrRport]’
/home/charl/julich/nest-simulator-install/include/nest/connector_model.h:164:25:   required from ‘nest::GenericConnectorModel<Connection

[100%] Linking CXX shared module nestml_6635cf53e5284717bffefafcf86c8804_module.so
[100%] Built target nestml_6635cf53e5284717bffefafcf86c8804_module_module
[100%] Built target nestml_6635cf53e5284717bffefafcf86c8804_module_module
Install the project...
-- Install configuration: ""
-- Installing: /tmp/nestml_target_2v1_i9cq/nestml_6635cf53e5284717bffefafcf86c8804_module.so


In [10]:
class AgentAction(enum.Enum):
    FAILURE = -1
    LEFT = 0
    RIGHT = 1

class SpikingAgent(Agent):
    cycle_period = 10.   # alternate between physics and SNN simulation with this cycle length [ms]
   
    def __init__(self, initial_state: Tuple[float,float,float,float], gamma) -> None:
        super().__init__(initial_state)
        self.gamma = gamma
        self.construct_neural_network()
        self.Q_left = 0.
        self.Q_right = 0.
        self.Q_left_prev = 0.
        self.Q_right_prev = 0.
        self.scale_n_output_spikes_to_Q_value = 0.1
        self.dopamine_left = 0.
        self.dopamine_right = 0.
        self.last_action_chosen = AgentAction.LEFT   # ?! choose first action randomly
        self.R = 1.  # reward -- always 1!
        self.p_explore = 0. # XXX: this is different from Liu&Pan
        print("XXXXXXXXX: init p_explore to 0")

        self.Wmax = 0.3
        self.Wmin = 0.005
        
    def get_state_neuron(self, state) -> int:
        idx = 0
        thresholds = [self.x_thresholds, self.theta_thresholds, self.v_thresholds, self.w_thresholds]
        for dim, val, thresh in zip(self.dimensions, state, thresholds):
            i = self.discretize(val,thresh)
            if i == -1:
                return -1
            idx = idx * dim + i

        return idx
    
    def get_state_from_id(self, idx) -> None:
        assert idx >= 0 and idx < len(self.input_population)
        state = [-1,-1,-1,-1]
        for i in reversed(range(len(state))):
            state[i] = idx % self.dimensions[i]
            idx = idx // self.dimensions[i]
        return tuple(state)
    
    def construct_neural_network(self):
        nest.ResetKernel()
        nest.local_num_threads = nest_local_num_threads
        nest.resolution = .01
        nest.Install(input_layer_module_name)   # makes the generated NESTML model available
        nest.Install(output_layer_module_name)   # makes the generated NESTML model available

        self.input_size = self.dimensions[0] * self.dimensions[1] * self.dimensions[2] * self.dimensions[3]
        self.input_population = nest.Create(input_layer_neuron_model_name, self.input_size)
        print("Input dimensions: " + str((self.dimensions[0], self.dimensions[1], self.dimensions[2], self.dimensions[3])))
        print("Size of the input population: " + str(self.input_size))
        
        self.output_population_left = nest.Create(output_layer_neuron_model_name, 10)
        self.output_population_right = nest.Create(output_layer_neuron_model_name, 10)
        
        self.spike_recorder_input = nest.Create("spike_recorder")
        nest.Connect(self.input_population, self.spike_recorder_input)

        self.multimeter_left = nest.Create('multimeter', 1, {'record_from': ['V_m']})
        nest.Connect(self.multimeter_left, self.output_population_left)
        self.multimeter_right = nest.Create('multimeter', 1, {'record_from': ['V_m']})
        nest.Connect(self.multimeter_right, self.output_population_right)

        syn_opts = {"synapse_model": output_layer_synapse_model_name,
                    "weight": 0.1 + nest.random.uniform(min=0.0, max=1.0) * 0.02,
                    "beta": 0.01,
                    "tau_tr_pre": 20., # [ms]
                    "tau_tr_post": 20.,  # [ms]
                    "wtr_max": 0.1,
                    "wtr_min": 0.,
                    "pre_trace_increment": 0.0001,
                    "post_trace_increment": -1.05E-7}
        
        nest.Connect(self.input_population, self.output_population_left, syn_spec=syn_opts)
        nest.Connect(self.input_population, self.output_population_right, syn_spec=syn_opts)

        syn = nest.GetConnections(source=self.input_population, target=self.output_population_right)
        self.syn_right = nest.GetConnections(source=self.input_population, target=self.output_population_right)
        self.syn_left = nest.GetConnections(source=self.input_population, target=self.output_population_left)

#         print("after network init:")
#         print("\tbeta = " + str(syn.beta))
#         print("\tbefore w = " + str(syn.w))
#         print("\tbefore wtr = " + str(syn.wtr))
    
        self.output_population_spike_recorder_left = nest.Create("spike_recorder")
        nest.Connect(self.output_population_left, self.output_population_spike_recorder_left)

        self.output_population_spike_recorder_right = nest.Create("spike_recorder")
        nest.Connect(self.output_population_right, self.output_population_spike_recorder_right)
        
        # set default values for prev_syn_wtr_right and left
        self.prev_syn_wtr_right = self.syn_right.wtr
        self.prev_syn_wtr_left = self.syn_left.wtr
        
        
    #stores important connections in a JSON, can be used to plot features of network
    def save_network(self):
        connection_dictionary = {}
        for input_neuron_id in range(len(self.input_population)):
            neuron = self.input_population[input_neuron_id]
            conn_left = nest.GetConnections(source=neuron, target=self.output_population_left)
            conn_right = nest.GetConnections(source=neuron, target=self.output_population_right)
            state = self.get_state_from_id(input_neuron_id) #state is a tuple of the corresponding bins for each variable indexed at 0
            connection_dictionary[str(state)] = {"neuron": neuron.get(),
                                            "connection_left": conn_left.get(),
                                            "connection_right": conn_right.get(),
                                            }
        #os.makedirs("/saved_networks", exist_ok=True)
        with open("saved_networks/network.json", "w") as f:
            json.dump(connection_dictionary, f, indent=4)

#     def choose_action(self, Q_left, Q_right) -> AgentAction:
#         # exploration
# #         if a.episode < 100:
# #             a.p_explore = 1.
        
#         if np.random.random() < a.p_explore:
#             # act randomly
#             return np.random.choice([AgentAction.LEFT, AgentAction.RIGHT])

#         p = np.exp(10.0*(Q_left - Q_right))
#         if np.random.random() < p:
#             return AgentAction.LEFT

#         return AgentAction.RIGHT

    def choose_action(self, Q_left, Q_right) -> AgentAction:
        r"""Purely deterministic action selection"""
        if Q_left > Q_right:
            return AgentAction.LEFT
        
        return AgentAction.RIGHT

    def compute_Q_values(self) -> None:
        r"""The output of the SNN is interpreted as the (scaled) Q values."""
        self.Q_left_prev = self.Q_left
        self.Q_right_prev = self.Q_right

        n_events_in_last_interval_left = self.output_population_spike_recorder_left.n_events
        n_events_in_last_interval_right = self.output_population_spike_recorder_right.n_events
#         print("n_events_in_last_interval_left = " + str(n_events_in_last_interval_left))
#         print("n_events_in_last_interval_right = " + str(n_events_in_last_interval_right))
        self.Q_left = self.scale_n_output_spikes_to_Q_value * n_events_in_last_interval_left
        self.Q_right = self.scale_n_output_spikes_to_Q_value * n_events_in_last_interval_right

    # update Q_value using TD-Error with previous Q_value and reward = 0
    # cooldown_time in case the SNN doesn't need 40ms to update
    def failure_reset(self) -> None:
#         print("In SpikingAgent::failure_reset()")
        # if for some reason the simulation terminates super fast
        if self.Q_left_prev == None and self.Q_right_prev == None:
            return
        # what would we mean by that? negative dopamine is biologically inaccurate
        # inhibitory neuromodulators?
#         if self.choose_action(self.Q_left_prev, self.Q_right_prev) == AgentAction.RIGHT:
#             syn = nest.GetConnections(source=self.input_population, target=self.output_population_right)
#             syn.n = -self.Q_right
#         else:
#             syn = nest.GetConnections(source=self.input_population, target=self.output_population_left)
#             syn.n = -self.Q_left

        TD = -self.Q_new
         
        if self.last_action_chosen == AgentAction.RIGHT:
#             print("Chosen action = RIGHT")
            syn = nest.GetConnections(source=self.input_population, target=self.output_population_right)
#             print("\tbeta = " + str(syn.beta))
#             print("\tbefore w = " + str(syn.w))
#             print("\tbefore wtr = " + str(syn.wtr))
#             print("\tbefore prev_syn_wtr_right = " + str(self.prev_syn_wtr_right))
#             print("\tbefore TD = " + str(TD))
            syn.w += np.array(syn.beta) * TD * np.array(self.prev_syn_wtr_right)
 

#             print("\tafter w = " + str(syn.w))
        else:
            assert self.last_action_chosen == AgentAction.LEFT
#             print("Chosen action = LEFT")
            syn = nest.GetConnections(source=self.input_population, target=self.output_population_left)
#             print("\tbeta = " + str(syn.beta))
#             print("\tbefore w = " + str(syn.w))
#             print("\tbefore wtr = " + str(syn.wtr))
#             print("\tbefore prev_syn_wtr_right = " + str(self.prev_syn_wtr_right))
#             print("\tbefore TD = " + str(TD))
            syn.w += np.array(syn.beta) * TD * np.array(self.prev_syn_wtr_left)
#             print("\tafter w = " + str(syn.w))

        syn.w = np.minimum(syn.w, self.Wmax)
        syn.w = np.maximum(syn.w, self.Wmin)
            
        self.episode += 1
        self.p_explore *= .99    # decay exploration probability -- from Liu&Pan code

    def update(self, next_state: Tuple[float,float,float,float]) -> Tuple[int, dict]:

        #Reset all spike recorders and multimeters
        #self.multimeter_left.n_events = 0
        #self.multimeter_right.n_events = 0
        #self.spike_recorder_input.n_events = 0
        self.output_population_spike_recorder_left.n_events = 0
        self.output_population_spike_recorder_right.n_events = 0

        # make the correct input neuron fire
        self.input_population.firing_rate = 0.
        neuron_id = self.get_state_neuron(next_state)
        
        # if state was a failure
        if neuron_id == -1:
#             self.failure_reset(SpikingAgent.cycle_period)
            return AgentAction.FAILURE, None
        
        self.input_population[neuron_id].firing_rate = 5000.    # [s⁻¹]

        # simulate for one cycle
        nest.Simulate(SpikingAgent.cycle_period)
        
        self.compute_Q_values()

        # set new dopamine concentration on the synapses
        # PROBLEM: HOW DO WE HANDLE FAILURE? The physics simulation immediately resets after it.
        # Perhaps run the simulation without spiking to let the weights update? (BVogler)

        self.Q_new = max(self.Q_left, self.Q_right)
        
        if self.last_action_chosen == AgentAction.LEFT:
            Q_old = self.Q_left_prev
        elif self.last_action_chosen == AgentAction.RIGHT:
            Q_old = self.Q_right_prev
        else:
            assert self.last_action_chosen == AgentAction.FAILURE
        
        TD = self.gamma * self.Q_new + self.R - Q_old
         
        if self.last_action_chosen == AgentAction.RIGHT:
#             print("Chosen action = RIGHT")
#             print("\tbeta = " + str(syn.beta))
#             print("\tbefore w = " + str(syn.w))
#             print("\tbefore wtr = " + str(syn.wtr))
#             print("\tbefore prev_syn_wtr_right = " + str(self.prev_syn_wtr_right))
#             print("\tbefore TD = " + str(TD))
            self.syn_right.w += np.array(self.syn_right.beta) * TD * np.array(self.prev_syn_wtr_right)
            self.syn_right.w = np.minimum(self.syn_right.w, self.Wmax)
            self.syn_right.w = np.maximum(self.syn_right.w, self.Wmin)
#             print("\tafter w = " + str(syn.w))
        else:
#             print("Chosen action = LEFT")
            assert self.last_action_chosen == AgentAction.LEFT
#             print("\tbeta = " + str(syn.beta))
#             print("\tbefore w = " + str(syn.w))
#             print("\tbefore wtr = " + str(syn.wtr))
#             print("\tbefore prev_syn_wtr_right = " + str(self.prev_syn_wtr_right))
#             print("\tbefore TD = " + str(TD))
            self.syn_left.w += np.array(self.syn_left.beta) * TD * np.array(self.prev_syn_wtr_left)
            self.syn_left.w = np.minimum(self.syn_left.w, self.Wmax)
            self.syn_left.w = np.maximum(self.syn_left.w, self.Wmin)
#             print("\tafter w = " + str(syn.w))
            
#             fig,ax=plt.subplots()
#             ax.plot(np.arange(1200), self.prev_syn_wtr_left)
#             import uuid
#             fig.savefig("/tmp/weights_nest" + str(uuid.uuid4()) + ".png")

        self.last_action_chosen = self.choose_action(self.Q_left, self.Q_right)
            
        return self.last_action_chosen            
            
            
    def save_prev_syn_wtr(self):
        self.prev_syn_wtr_right = self.syn_right.wtr
        self.prev_syn_wtr_left = self.syn_left.wtr
        
#         # update Q_value using TD-Error with previous Q_value and reward = 1
#         if self.Q_left_prev != None and self.Q_right_prev != None:
#             if self.choose_action(self.Q_left_prev, self.Q_right_prev) == ...:
#                 last_action_chosen = AgentAction....
#                 syn = nest.GetConnections(source=self.input_population, target=self.output_population_right)
#                 syn.n = self.gamma * self.Q_right + R - self.Q_right_prev
#                 self.dopamine_right = syn.n[0] #for displaying stats
#             else:
#                 syn = nest.GetConnections(source=self.input_population, target=self.output_population_left)
#                 syn.n = self.gamma * self.Q_left + R - self.Q_left_prev
#                 self.dopamine_left = syn.n[0] #for displaying stats
        
#         # 0 if action is "left", else 1


# Executing spiking version

The main loop looks like this: for every iteration of the loop (for every "cycle" or "step"):

- set the rate of the input neurons to the current state of the system
- run the SNN with this input state s_n for a period of time (cycle time, in BVogler's thesis: 40 ms)
- obtain the Q(sn, a) values, by counting nr of spikes in output population over this cycle period
- choose action $a_n$ on the basis of Q-values
- run the environment for the same cycle time (40 ms) to obtain next state $s_{n+1}$
- compute reward on the basis of the last taken action (????)

In [ ]:
%pdb
import sys

r = Renderer(1200, 800, 600, 500, 400)
clock = pg.time.Clock()
running = True

episode_number_log = []
steps_per_episode_log = []
episode_number_times = []
episode_number_at_time = []

action_taken_times = []
action_taken = []
p_explore_log = []
epoch_log = []

p = Physics(0, (np.random.rand() - 1) / 10)

a = SpikingAgent(p.get_state(), gamma=0.98)
syn_to_left = nest.GetConnections(source=a.input_population, target=a.output_population_left)
syn_to_right = nest.GetConnections(source=a.input_population, target=a.output_population_right)


plot = Spiking_PlotRenderer()

steps_per_episode = 0

max_n_steps_per_episode = 1_000_000

toggle_sim = True
plot_spikes = True
stepping_sim = False
while running:
    start_time_main_loop_iteration = time.time()

    steps_per_episode += 1
    force = 0
    mouse_x = None
   
    # poll for events
    for event in pg.event.get():
        if event.type == pg.QUIT:
            running = False
            pg.quit()
            sys.exit()
            quit()
        elif event.type == pg.MOUSEBUTTONDOWN:
            mouse_x = r.get_relative_mouse_x(pg.mouse.get_pos()[0])
        elif event.type == pg.KEYDOWN:
            #controls if simulation should be shown or not
            toggle_sim ^= pg.key.get_pressed()[pg.K_1]
            #on button press plots the current spikes
            plot_spikes ^= pg.key.get_pressed()[pg.K_2]
            #on button press stores the network in ./saved_networks/network.json
            if pg.key.get_pressed()[pg.K_3]:
                a.save_network()
            #toggles step-by-step simulation, updating now by pressing space
            stepping_sim ^= pg.key.get_pressed()[pg.K_4]

    if stepping_sim:
        toggle_sim = True
        plot_spikes = True
        next_step = False
        while not next_step and stepping_sim:
            pg.event.wait()
            stepping_sim ^= pg.key.get_pressed()[pg.K_4]
            next_step ^= pg.key.get_pressed()[pg.K_SPACE]

    start_time_agent_update = time.time()
    global action
    action = a.update(p.get_state())
    assert action != AgentAction.FAILURE

    if action == AgentAction.RIGHT:
        force = 10
    elif action == AgentAction.LEFT:
        force = -10
    else:
        assert False, "Unknown action returned"
    
    end_time_agent_update = time.time()

    action_taken_times.append(nest.biological_time)
    action_taken.append(action)
    p_explore_log.append(a.p_explore)
    epoch_log.append(a.episode)

    if not a.episode in episode_number_log:
        episode_number_log.append(a.episode)
        steps_per_episode_log.append(steps_per_episode)
    else:
        idx = episode_number_log.index(a.episode)
        steps_per_episode_log[idx] = max(steps_per_episode_log[idx], steps_per_episode)

    start_time_physics_update = time.time()
    theta, x = p.update(force, mouse_x)
    end_time_physics_update = time.time()
    
    if a.get_state_neuron(p.get_state()) == -1 or steps_per_episode > max_n_steps_per_episode:
        # failure state
        a.failure_reset()
        p.reset()
        steps_per_episode = 0

        if plot_spikes:
            # do the plot
            syn_left = nest.GetConnections(source=a.input_population, target=a.output_population_left)
            syn_right = nest.GetConnections(source=a.input_population, target=a.output_population_right)

            weights_matrix_left, _, _ = create_weight_matrix(syn_left)
            weights_matrix_right, _, _ = create_weight_matrix(syn_right)

            # plot data, passed onto Spiking_Plot_Renderer()
            plot_data = {
                "weights_left": weights_matrix_left,
                "weights_right": weights_matrix_right,
                "input_spikes": nest.GetStatus(a.spike_recorder_input, keys="events")[0],
                "output_spikes_left": nest.GetStatus(a.output_population_spike_recorder_left, keys="events")[0],
                "output_spikes_right": nest.GetStatus(a.output_population_spike_recorder_right, keys="events")[0],
                "multimeter_right_events": a.multimeter_right.get("events"),
                "multimeter_left_events": a.multimeter_left.get("events"),
                "n_input_neurons": a.input_size
            }

            plot_data["action_taken_times"] = action_taken_times
            plot_data["action_taken"] = action_taken
            plot_data["p_explore_log"] = p_explore_log
            plot_data["epoch_log"] = epoch_log
            plot_data["episode_duration"] = a.cycle_period
            plot_data["episode_number_log"] = episode_number_log
            plot_data["steps_per_episode_log"] = steps_per_episode_log
            
            plot.update(plot_data)

    else:
        a.save_prev_syn_wtr()
    

    #
    #    reset synaptic state
    #
    
    for _syn in [syn_to_left, syn_to_right]:
        _syn.wtr = 0.
        _syn.pre_trace = 0.
        #_syn.post_trace = 0. # need to do this in postsyn. neuron partner instead! See the next two lines
    
    a.output_population_left.post_trace__for_neuromodulated_stdp_synapse_nestml = 0.
    a.output_population_right.post_trace__for_neuromodulated_stdp_synapse_nestml = 0.

    end_time_main_loop_iteration = time.time()
    
   
    #
    #    render
    #
    
    if toggle_sim:
        r.draw_clear()
        r.draw_ground(0.2, "grey")
        r.draw_car(x)
        r.draw_pole(x, theta, 2*p.l, 0.02)
        r.draw_stats(theta*180/np.pi, p.w*180/np.pi, x, p.v, a.get_episode(),
                     a.output_population_spike_recorder_left.n_events, 
                     a.output_population_spike_recorder_right.n_events,
                     a.dopamine_left,
                     a.dopamine_right,
                     action)
        r.display()

        clock.tick(50)  # limits FPS to 50
        
    #
    #    print performance stats
    #
    
    if steps_per_episode == 1:
        time_agent_update = end_time_agent_update - start_time_agent_update
        print(f"Agent update took: {time_agent_update:.3f} s")
        time_physics_update = end_time_physics_update - start_time_physics_update
        print(f"Phyics update took: {time_physics_update:.3f} s")
        time_main_loop_iteration = end_time_main_loop_iteration - start_time_main_loop_iteration
        print(f"Main loop took: {time_main_loop_iteration:.3f} s")


Automatic pdb calling has been turned ON
Dimension of input space: (3, 6, 3, 3)
Input dimensions: (3, 6, 3, 3)
Size of the input population: 162
XXXXXXXXX: init p_explore to 0


DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x7fbee4559d50>
DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbee458c190> <matplotlib.colors.Normalize object at 0x7fbee458c190>
DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x7fbee4559d50>
/tmp/ipykernel_1380245/1348692311.py:114: UserWarning:Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbf28cfbcd0> <matplotlib.colors.Normalize object at 0x7fbee458c190>
DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x7fbf293d8fd0>
DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbf293d9890> <matplotlib.colors.Normalize object at 0x7fbf28cfbcd0>
DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x7fbf293d8890>
DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbf28f52090> <matplotlib.colors.Normalize object at 0x7fbf293d9890>

DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbf28d07c50> <matplotlib.colors.Normalize object at 0x7fbf29b5c990>
DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x7fbed43c21d0>
DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbee45a32d0> <matplotlib.colors.Normalize object at 0x7fbf28d07c50>
DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x7fbed43f2d50>
DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbf28d83290> <matplotlib.colors.Normalize object at 0x7fbee45a32d0>
DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x7fbed43a4b10>
DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbed43a6f50> <matplotlib.colors.Normalize object at 0x7fbf28d83290>
DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x7fbed43a4cd0>
DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbee458c490> <matplotlib.colors.Normalize object at 0x7fbed43a6f50>
DEBUG:locator: <m

DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbf29808490> <matplotlib.colors.Normalize object at 0x7fbed4148390>
DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x7fbf28e55210>
DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbed412c050> <matplotlib.colors.Normalize object at 0x7fbf29808490>
DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x7fbed4261610>
DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbf28ae1990> <matplotlib.colors.Normalize object at 0x7fbed412c050>
DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x7fbed4262dd0>
DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbf28cb3b10> <matplotlib.colors.Normalize object at 0x7fbf28ae1990>
DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x7fbf28fe2a90>
DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbf28f86c50> <matplotlib.colors.Normalize object at 0x7fbf28cb3b10>
DEBUG:locator: <m

DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbf28d151d0> <matplotlib.colors.Normalize object at 0x7fbed420b590>
DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x7fbf28f4efd0>
DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbed434fe90> <matplotlib.colors.Normalize object at 0x7fbf28d151d0>
DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x7fbf28d54c10>
DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbf28d55dd0> <matplotlib.colors.Normalize object at 0x7fbed434fe90>
DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x7fbf28e7fa90>
DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbf293c4710> <matplotlib.colors.Normalize object at 0x7fbf28d55dd0>
DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x7fbf28d2b4d0>
DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbf28e3f250> <matplotlib.colors.Normalize object at 0x7fbf293c4710>
DEBUG:locator: <m

...

DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbed4290a10> <matplotlib.colors.Normalize object at 0x7fbf28f8f090>
DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x7fbed42c3490>
DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbed4210490> <matplotlib.colors.Normalize object at 0x7fbed4290a10>
DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x7fbf28eff490>
DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbed41423d0> <matplotlib.colors.Normalize object at 0x7fbed4210490>
DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x7fbee46086d0>
DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbed42b0650> <matplotlib.colors.Normalize object at 0x7fbed41423d0>
DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x7fbf28ee8e10>
DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbed422f650> <matplotlib.colors.Normalize object at 0x7fbed42b0650>
DEBUG:locator: <m

DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbf29c3e3d0> <matplotlib.colors.Normalize object at 0x7fbf28efdb10>
DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x7fbee46397d0>
DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbf29c3f510> <matplotlib.colors.Normalize object at 0x7fbf29c3e3d0>
DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x7fbed428bcd0>
DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbed431a090> <matplotlib.colors.Normalize object at 0x7fbf29c3f510>
DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x7fbed428b090>
DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbee4588ad0> <matplotlib.colors.Normalize object at 0x7fbed431a090>
DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x7fbf29806410>
DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbee458b990> <matplotlib.colors.Normalize object at 0x7fbee4588ad0>
DEBUG:locator: <m

DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbe9fbe7690> <matplotlib.colors.Normalize object at 0x7fbf291d8e90>
DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x7fbf28c91090>
DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbf28ae3990> <matplotlib.colors.Normalize object at 0x7fbe9fbe7690>
DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x7fbed4369810>
DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbf28c9f710> <matplotlib.colors.Normalize object at 0x7fbf28ae3990>
DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x7fbed436ad50>
DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbf28cb3490> <matplotlib.colors.Normalize object at 0x7fbf28c9f710>
DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x7fbf28ffc7d0>
DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbed4258610> <matplotlib.colors.Normalize object at 0x7fbf28cb3490>
DEBUG:locator: <m

DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbf29c1e950> <matplotlib.colors.Normalize object at 0x7fbf28e57c90>
DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x7fbee45de290>
DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbee4614150> <matplotlib.colors.Normalize object at 0x7fbf29c1e950>
DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x7fbf29c1f690>
DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbf28c8de10> <matplotlib.colors.Normalize object at 0x7fbee4614150>
DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x7fbed4367cd0>
DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbf29186c50> <matplotlib.colors.Normalize object at 0x7fbf28c8de10>
DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x7fbed43646d0>
DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbf29108490> <matplotlib.colors.Normalize object at 0x7fbf29186c50>
DEBUG:locator: <m

DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbee4562090> <matplotlib.colors.Normalize object at 0x7fbf28b64690>
DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x7fbf29130e10>
DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbee4561850> <matplotlib.colors.Normalize object at 0x7fbee4562090>
DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x7fbf29133390>
DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbee4562ed0> <matplotlib.colors.Normalize object at 0x7fbee4561850>
DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x7fbed4335210>
DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbf297eaa50> <matplotlib.colors.Normalize object at 0x7fbee4562ed0>
DEBUG:locator: <matplotlib.ticker.AutoLocator object at 0x7fbee4606d90>
DEBUG:colorbar update normal <matplotlib.colors.Normalize object at 0x7fbf28e9c290> <matplotlib.colors.Normalize object at 0x7fbf297eaa50>
DEBUG:locator: <m

## Citations

[1] Liu Y, Pan W. Spiking Neural-Networks-Based Data-Driven Control. Electronics. 2023; 12(2):310. https://doi.org/10.3390/electronics12020310 

## Acknowledgements

The authors would like to thank Prof. Wei Pan and Dr. Yuxiang Liu for kindly providing ...